In [53]:
from collections import Counter
from functools import partial

import math, random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

def step_function(x):
    return 1 if x >= 0 else 0

def perceptron_output(weights, bias, x):
    """returns 1 if the perceptron 'fires', 0 if not"""
    return step_function(np.dot(weights, x) + bias)

def sigmoid(t):
    return 1 / (1 + math.exp(-t))

def neuron_output(weights, inputs):
    return sigmoid(np.dot(weights, inputs))

def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""

    outputs = []

    for layer in neural_network:

        input_with_bias = input_vector + [1]             # add a bias input
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output

    return outputs

def backpropagate(network, input_vector, target):

    hidden_outputs, outputs = feed_forward(network, input_vector)

    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]

    print(sum(output_deltas))
    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            output_neuron[j] -= output_deltas[i] * hidden_output

    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                      np.dot(output_deltas, [n[i] for n in network[-1]])
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

def patch(x, y, hatch, color):
    """return a matplotlib 'patch' object with the specified
    location, crosshatch pattern, and color"""
    return matplotlib.patches.Rectangle((x - 0.5, y - 0.5), 1, 1,
                                        hatch=hatch, fill=False, color=color)


def show_weights(neuron_idx):
    weights = network[0][neuron_idx]
    abs_weights = [abs(weight) for weight in weights]

    grid = [abs_weights[row:(row+5)] # turn the weights into a 5x5 grid
            for row in range(0,25,5)] # [weights[0:5], ..., weights[20:25]]

    ax = plt.gca() # to use hatching, we'll need the axis

    ax.imshow(grid, # here same as plt.imshow
              cmap=matplotlib.cm.binary, # use white-black color scale
              interpolation='none') # plot blocks as blocks

    # cross-hatch the negative weights
    for i in range(5): # row
        for j in range(5): # column
            if weights[5*i + j] < 0: # row i, column j = weights[5*i + j]
                # add black and white hatches, so visible whether dark or light
                ax.add_patch(patch(j, i, '/', "white"))
                ax.add_patch(patch(j, i, '\\', "black"))
    plt.show()



In [57]:
if __name__ == "__main__":

    raw_digits = [
          """11111
             1...1
             1...1
             1...1
             11111""",

          """..1..
             ..1..
             ..1..
             ..1..
             ..1..""",

          """11111
             ....1
             11111
             1....
             11111""",

          """11111
             ....1
             11111
             ....1
             11111""",

          """1...1
             1...1
             11111
             ....1
             ....1""",

          """11111
             1....
             11111
             ....1
             11111""",

          """11111
             1....
             11111
             1...1
             11111""",

          """11111
             ....1
             ....1
             ....1
             ....1""",

          """11111
             1...1
             11111
             1...1
             11111""",

          """11111
             1...1
             11111
             ....1
             11111"""]

    def make_digit(raw_digit):
        return [1 if c == '1' else 0
                for row in raw_digit.split("\n")
                for c in row.strip()]

    inputs = list(map(make_digit, raw_digits))

    targets = [[1 if i == j else 0 for i in range(10)]
               for j in range(10)]

    random.seed(0)   # to get repeatable results
    input_size = 25  # each input is a vector of length 25
    num_hidden = 5   # we'll have 5 neurons in the hidden layer
    output_size = 10 # we need 10 outputs for each input

    # each hidden neuron has one weight per input, plus a bias weight
    hidden_layer = [[random.random() for __ in range(input_size + 1)]
                    for __ in range(num_hidden)]

    # each output neuron has one weight per hidden neuron, plus a bias weight
    output_layer = [[random.random() for __ in range(num_hidden + 1)]
                    for __ in range(output_size)]

    # the network starts out with random weights
    network = [hidden_layer, output_layer]

    # 10,000 iterations seems enough to converge
    for __ in range(1000):
        for input_vector, target_vector in zip(inputs, targets):
            backpropagate(network, input_vector, target_vector)

    

0.5344198244655093
0.6235226966170309
0.8542054749021974
0.9667659842614225
0.7531615548934627
0.6033072466923627
0.5868420381525405
0.6316967587198633
0.5228842917366973
0.41622291451743454
0.31738253144343526
0.2665242407638393
0.13915517428680557
0.10408484969510463
0.12963700267306016
0.09731921886491784
0.07769455767398353
0.07050378816007655
0.03954064661864507
0.061118570258979904
0.04592891534008063
0.05002045463403904
0.03162931027729354
0.027826687377010384
0.03348295759966915
0.02987589932778486
0.026544082509118973
0.02652055024925657
0.01694630548779899
0.02378222026285841
0.019150544371050883
0.023836435312259107
0.01471490242129144
0.013122072605150938
0.015552520029333768
0.014312110183482881
0.012969345763368698
0.01357327343948812
0.008890599867151879
0.012162039340414876
0.009982357997639336
0.0140825914272137
0.008098662653141986
0.0071848646847785455
0.008457156357424108
0.00786375658160601
0.007122652582894973
0.00781064159480223
0.005060848062351283
0.00686856962

-0.00038547478527510906
0.0003470301801409021
-3.28053503726531e-05
-0.0002472345754854985
-0.00041675185603097693
0.00033691184159684653
0.0006332505951950843
-6.912607945540675e-05
-0.00014932689736237362
-4.652342899041234e-05
-0.0003837974749308715
0.0003538834079731589
-2.7018023010979587e-05
-0.0002460356624299706
-0.00041808860506280764
0.0002930156912394691
0.0006482050787277377
-6.536962170930298e-05
-0.0001462421050154811
-4.069115080575659e-05
-0.00038207708507525997
0.000360964643470448
-2.1067227830602088e-05
-0.0002448098985431066
-0.00041947297858989374
0.000247679780556306
0.0006636504668928152
-6.14699684642582e-05
-0.00014305368770180758
-3.467698049359341e-05
-0.00038031061577700146
0.00036828551588451944
-1.4944595258400105e-05
-0.00024355551067044984
-0.0004209067114198609
0.00020082332740989188
0.0006796125619784052
-5.7418641080662566e-05
-0.00013975533739285766
-2.8471108226031834e-05
-0.0003784948419997408
0.00037585843461054
-8.641201203666779e-06
-0.000242270

0.0005840427026229042
-0.0012089864095473305
-0.05984715077077419
0.0199341713300202
0.007101182069055663
0.004937260324297626
0.005819623945906012
0.0018958323259718537
0.007668942504826318
0.004669246022481975
0.000509575302568073
-0.0013309932352938633
-0.05953953468963075
0.020098281910043557
0.006915546304818381
0.00481002841370883
0.0057772871527915675
0.0017750080380919834
0.00784061690464528
0.004702545966485494
0.0004034365794948619
-0.0014656513427180762
-0.05778425027008767
0.019835457021856746
0.006511051069364676
0.0045360385089799875
0.005614755940418842
0.00158341929255434
0.00790956563218068
0.0046652219908856
0.0002693972647115836
-0.0016101629067898898
-0.054776629746354076
0.019212148698833732
0.005924479929840401
0.004137124135969596
0.005348263972578841
0.001331465492717324
0.007883128850708292
0.004564689372779625
0.00011220336942233122
-0.0017618303908116233
-0.05078840204049733
0.01831769965479306
0.005203531299879386
0.0036420857917780474
0.004998991483894867
0

0.028854290526056234
-0.0010276002985686455
-0.0043712568012521415
-0.0037811646646887217
-0.0025095416222467564
-0.0035461798398423693
-0.000544690977241454
-0.0017400009199085497
-0.0027683743508286757
-0.003132779003422719
0.02875771089645425
-0.001041944301800499
-0.004343609366395348
-0.0037577002509570174
-0.002501700343032766
-0.0035269471497120924
-0.0005596808443809878
-0.0017437766909782371
-0.0027557004217465125
-0.003114249145907943
0.028659780491024217
-0.001055631330695887
-0.0043160964170709595
-0.00373432286440959
-0.0024937165863743423
-0.003507773450622311
-0.0005742006039283062
-0.001747217056003462
-0.0027430356754993696
-0.0030958437287384264
0.028560621987673843
-0.0010686886137820685
-0.004288725841757922
-0.0037110404983706734
-0.0024856018972732605
-0.0034886654238398254
-0.000588270502136121
-0.0017503368709410024
-0.0027303856222848644
-0.0030775651645951266
0.028460350476272336
-0.0010811420371682236
-0.004261504773338122
-0.003687860442368732
-0.00247736709

-0.0016739722059361598
-0.002121256649444056
-0.0022722783848276255
0.022857152738415987
-0.0012658290252610307
-0.0030473818703086013
-0.0026354980031986115
-0.001991655328227192
-0.0025926987173301884
-0.0010265067932011948
-0.0016705347745302607
-0.0021120324081460445
-0.002260839799612861
0.022766488347310626
-0.001264465639036802
-0.0030301365330220407
-0.002620336827238284
-0.001983416563842995
-0.0025798899088416843
-0.0010320467221012252
-0.0016670702747094935
-0.002102869198856633
-0.002249493810913413
0.02267637732990003
-0.0012630188166518341
-0.0030130348351962
-0.0026052964600868905
-0.00197521249709623
-0.002567178716333526
-0.0010375947593208642
-0.0016635798588971054
-0.0020937664229838115
-0.0022382392405939713
0.022586820053178146
-0.001261490870870548
-0.002996075038225056
-0.002590375498839982
-0.0019670430889575834
-0.0025545641022601826
-0.001043154163058545
-0.0016600646282291034
-0.002084723473984307
-0.0022270749164907556
0.02249781665967743
-0.0012598840268024

...

0.03481336150046712
0.0021401858143438885
0.0015139063265191145
0.0014845029934651074
0.0012014148994773915
0.0009381520952774355
-0.053280968681269185
0.004840119881964038
0.004497832814426145
0.0042143608505826515
0.03490616649864605
0.002140577805979765
0.0015162280313771488
0.0014858882403824061
0.0012029992714811567
0.0009409281080197908
-0.05341113477887495
0.004848396252791488
0.004506135809622933
0.004222534472368594
0.03499841179006086
0.002140973620991171
0.0015185532173320054
0.0014872747571895838
0.0012045910302736895
0.0009437056714410533
-0.05354065956639147
0.004856635946232343
0.004514399093321228
0.004230666305472829
0.035090103136908565
0.002141373157687851
0.0015208817999125147
0.0014886624851805606
0.0012061900942869352
0.0009464847228238303
-0.053669549704743476
0.004864839577148259
0.004522623288918684
0.004238756981005603
0.03518124630927573
0.002141776305701476
0.0015232136871742123
0.0014900513576284538
0.0012077963752620945
0.0009492651938344315
-0.05379781177

0.005357288531073279
0.005011084517646425
0.004714660669880124
0.04038503624026443
0.0021679256618082118
0.0016815094047961283
0.0015799492353076906
0.0013263199188771092
0.0011356262231292926
-0.061337575158142255
0.0053637214453376705
0.005017411865138183
0.0047207776889716686
0.040450648665660144
0.0021681372233784742
0.001683710683176208
0.0015810852011913963
0.001328080977092352
0.0011382339518434858
-0.06143447773967823
0.005370133289008913
0.005023717719224816
0.004726873203535736
0.04051603228871527
0.002168339658490623
0.0016859052506846516
0.0015822126134066843
0.001329839522311263
0.0011408355088617641
-0.061531052817342446
0.005376524125292639
0.005030002178411042
0.00473294733947827
0.04058119002307849
0.002168532819085342
0.0016880930116123412
0.0015833313630947473
0.0013315954539467548
0.0011434308210178656
-0.0616273029895027
0.0053828940169384125
0.005036265340345236
0.004739000221533712
0.040646124739826095
0.0021687165587792948
0.001690273872363559
0.0015844413431267

0.0016331140982632532
0.0014352532111242492
0.0012943263508549928
-0.06721056549278986
0.005749989216935713
0.005397094599937424
0.005087471948856769
0.044441794474143634
0.0021551678190698392
0.0018126427822555986
0.0016335070856603514
0.0014367032554354637
0.0012964270887841191
-0.06729009773782622
0.005755143898740102
0.005402176235411116
0.00509239031269643
0.04449673862780988
0.002154481226514385
0.0018142601321445122
0.0016338872480494496
0.0014381475232467592
0.0012985197032063574
-0.06736942739419251
0.0057602825912214375
0.005407243026908329
0.005097295057984157
0.04455157863682312
0.002153777689802799
0.0018158680847856767
0.0016342545529521146
0.0014395860065964904
0.0013006041978482303
-0.06744855584754585
0.005765405394562209
0.005412295087679972
0.005102186309193476
0.04460631556050874
0.0021530570989957247
0.0018174666412452793
0.0016346089683657063
0.0014410186983487873
0.0013026805770247427
-0.06752748447652199
0.0057705124098677485
0.0054173325317028315
0.005107064191

0.0020588553761577023
0.0019036712934123162
0.001627596639257196
0.0015240489810144538
0.0014238881329125012
-0.07243014346682952
0.006082991125851463
0.005728355704672811
0.005410774993029612
0.048135283203128924
0.0020566281837646767
0.0019046578413496676
0.0016270150032450127
0.0015251055442276496
0.0014254408820955902
-0.07249863093327569
0.006087323333675054
0.005732720573524405
0.00541508937309519
0.048184979950667624
0.0020543715832156643
0.0019056359807456211
0.001626418904406868
0.0015261572222311434
0.0014269863920452035
-0.07256701533714469
0.006091649746889586
0.005737081408848946
0.005419401725478138
0.04823463117776451
0.002052085271301329
0.00190660573104149
0.0016258083307881272
0.0015272040424056874
0.0014285246697433446
-0.0726352984854748
0.006095970577680442
0.005741438428879572
0.005423712284061253
0.04828423774487196
0.0020497689391162973
0.0019075671116989094
0.0016251832712212925
0.00152824603242906
0.0014300557214422215
-0.0727034822142243
0.006100286041420255


0.001513939555833681
-0.0771280761057631
0.006388823775967774
0.006040955096115033
0.005726129146554236
0.05157402855570958
0.001802449311073432
0.0019541057917589356
0.001551961955348841
0.001588739327344461
0.001514883673117696
-0.07719774615363219
0.0063936184598639865
0.006045911722266734
0.0057312571759738275
0.05162434884508725
0.0017968649585100183
0.0019545613192442253
0.0015505091059692198
0.0015895307779561114
0.0015158130482911657
-0.07726770138554903
0.0063984439082821355
0.006050900646854776
0.0057364234139289216
0.05167479760316303
0.0017912026271158493
0.001955011462670197
0.001549054355408622
0.001590319791955246
0.0015167271947371646
-0.07733795912192361
0.006403301522673072
0.006055923230337407
0.005741629402576311
0.05172538246945868
0.001785460888217922
0.0019554563845499816
0.001547598598030801
0.0015911064036981098
0.001517625594843532
-0.07740853762818722
0.006408192771831155
0.006060980897050802
0.005746876758260999
0.0517761114931371
0.0017796382729693954
0.001

In [68]:
def predict(input):
        return feed_forward(network, input)[-1]

zero_digit = [
    1,1,1,1,1,
    1,0,0,0,1,
    1,0,0,0,1,
    1,0,0,0,1,
    1,1,1,1,1
             ]

two_digit = [
    1,1,1,1,1,
    0,0,0,0,1,
    1,1,1,1,1,
    1,0,0,0,0,
    1,1,1,1,1
]


pred = predict(zero_digit)

print(np.argmax(pred))

print(pred)

9
[0.0880433033317668, 0.00020785762213645033, 0.08782467190248139, 0.09343254781034815, 0.096215155714184, 0.10531235629270266, 0.11293066139740614, 0.08682151842370203, 0.12097073945111557, 0.1304331068671273]


In [ ]:
show_weights(2)

In [69]:
for i, input in enumerate(inputs):
    outputs = predict(input)
    print(i, [round(p,2) for p in outputs])

    print(""".@@@.
...@@
..@@.
...@@
.@@@.""")
    print([round(x, 2) for x in
          predict(  [0,1,1,1,0,    # .@@@.
                     0,0,0,1,1,    # ...@@
                     0,0,1,1,0,    # ..@@.
                     0,0,0,1,1,    # ...@@
                     0,1,1,1,0])]) # .@@@.
    print()

    print(""".@@@.
@..@@
.@@@.
@..@@
.@@@.""")
    print([round(x, 2) for x in
          predict(  [0,1,1,1,0,    # .@@@.
                     1,0,0,1,1,    # @..@@
                     0,1,1,1,0,    # .@@@.
                     1,0,0,1,1,    # @..@@
                     0,1,1,1,0])]) # .@@@.
    print()

0 [0.09, 0.0, 0.09, 0.09, 0.1, 0.11, 0.11, 0.09, 0.12, 0.13]
.@@@.
...@@
..@@.
...@@
.@@@.
[0.06, 0.0, 0.07, 0.07, 0.07, 0.07, 0.08, 0.12, 0.08, 0.09]

.@@@.
@..@@
.@@@.
@..@@
.@@@.
[0.08, 0.0, 0.09, 0.09, 0.09, 0.1, 0.11, 0.09, 0.12, 0.12]

1 [0.01, 0.85, 0.03, 0.02, 0.02, 0.02, 0.02, 0.32, 0.02, 0.02]
.@@@.
...@@
..@@.
...@@
.@@@.
[0.06, 0.0, 0.07, 0.07, 0.07, 0.07, 0.08, 0.12, 0.08, 0.09]

.@@@.
@..@@
.@@@.
@..@@
.@@@.
[0.08, 0.0, 0.09, 0.09, 0.09, 0.1, 0.11, 0.09, 0.12, 0.12]

2 [0.08, 0.0, 0.09, 0.09, 0.09, 0.1, 0.11, 0.09, 0.12, 0.12]
.@@@.
...@@
..@@.
...@@
.@@@.
[0.06, 0.0, 0.07, 0.07, 0.07, 0.07, 0.08, 0.12, 0.08, 0.09]

.@@@.
@..@@
.@@@.
@..@@
.@@@.
[0.08, 0.0, 0.09, 0.09, 0.09, 0.1, 0.11, 0.09, 0.12, 0.12]

3 [0.09, 0.0, 0.09, 0.09, 0.09, 0.1, 0.11, 0.09, 0.12, 0.13]
.@@@.
...@@
..@@.
...@@
.@@@.
[0.06, 0.0, 0.07, 0.07, 0.07, 0.07, 0.08, 0.12, 0.08, 0.09]

.@@@.
@..@@
.@@@.
@..@@
.@@@.
[0.08, 0.0, 0.09, 0.09, 0.09, 0.1, 0.11, 0.09, 0.12, 0.12]

4 [0.08, 0.0, 0.09, 0.09, 0.09